# Soil Audits
### Outputs
* Mismatches between dates in Smart Shseets and Soil tracker(Share Point)
* Duplicate APNs-soil only

In [33]:
import pandas as pd
import os
import glob
import numpy as np
import datetime as dt

In [34]:
# smart Sheets
path1 = r"C:\Users\jacque.trahan\Downloads\Central Division 2021 Debris Removal Tracker*"
file_type1 =".xlsx"
loadit1 = glob.glob(path1 + file_type1)
df1 = pd.read_excel(max(loadit1, key=os.path.getctime))

In [35]:
# Soil Share Point
path2 = r"C:\Users\jacque.trahan\Downloads\Soil Sampling Tracker*"
file_type2 = ".csv"
loadit2 = glob.glob(path2 + file_type2)
df2 = pd.read_csv(max(loadit2, key=os.path.getctime))

In [36]:
smart_sheets = df1.copy()
# smart_sheets.columns = map(str.upper, smart_sheets.columns)
smart_sheets = smart_sheets[['APN_ROW Segment','Street #','Street Name','Structural Status','County', 'Soil Sample', 'Soil Sample Results Rec\'d from Lab',
                             'Division']]
smart_sheets = smart_sheets[smart_sheets['Division'].isin(['Central Division'])]

smart_sheets.rename(columns={'Soil Sample':'Soil Sample_SS'}, inplace=True)

In [37]:
soil = df2.copy()
soil = soil[['APN', 'Street Number','Street Name', 'County', 'Date Collected','Date Shipped', 'Date Lab Recd', 'Date Results Recd', 'Division']]
soil = soil[soil['Division'].isin(['Central Division'])]

In [8]:
def compare(final, comp1, comp2):
    if final[comp1] == final[comp2]:
        return 1
    else:
        return 0

In [9]:
final = smart_sheets.merge(soil,
                          left_on='APN_ROW Segment',
                          right_on="APN",
                          how="outer",
                          suffixes=("_SS","_SOIL"))
final = final[['APN_ROW Segment','APN', 'Street #','Street Name','Structural Status','County', 'Soil Sample_SS',
               'Date Collected','Date Shipped','Date Lab Recd', 'Date Results Recd', 'Division_SS' ]]

In [10]:
final['Date Collected'] = pd.to_datetime(final['Date Collected']).dt.date.fillna(0)
final['Soil Sample_SS'] = pd.to_datetime(final['Soil Sample_SS']).dt.date.fillna(0)
final['Date Shipped']  = pd.to_datetime(final['Date Shipped'])
final['Date Results Recd']  = pd.to_datetime(final['Date Results Recd'])

In [11]:
this will compare the date columns
final.insert(final.columns.get_loc('Date Collected')+1, "Matches",
            final.apply(compare, comp1='Soil Sample_SS', comp2='Date Collected', axis=1))

In [12]:
final_style = final.style.set_properties(**{'background-color': 'lightblue'},
                                        subset=['Matches'])

# Duplicate APNs

In [15]:
# df2['APN'].value_counts().to_excel('Share Point Duplicate APNS Soil Central.xlsx')